<a href="https://colab.research.google.com/github/ICBI/BISR_Tutorials/blob/main/scRNA_seq/SCRNAseq_Seurat_Tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2 - single cell RNA-seq (scRNA-seq) data analysis of mouse samples from 10X

Authors: Krithika Bhuvaneshwar, Yuriy Gusev

Affiliation: Innovation Center For Biomedical Informatics (ICBI), and Biomedical Informatics Shared Resource (BISR) at Georgetown University Medical Center (GUMC)

***More about our research work:***
* *ICBI: https://icbi.georgetown.edu*
* *BISR: https://icbi.georgetown.edu/bisr/ and https://lombardi.georgetown.edu/research/sharedresources/bbsr/*


## General Steps in analysis
Based on the workflow mentioned in the e-book "Orchestrating Single-Cell Analysis with Bioconductor"
* Removal of low-quality cells
* Normalization and log-transformation
* Modeling of the mean-variance trend across genes
* A principal components analysis on the highly variable genes
* Clustering with graph-based methods
* Dimensionality reductions (t-SNE/UMAP)
* Marker detection for each cluster
* Make custom cell selections and detect markers for this selection
* Cell type annotation for each cluster across user selected reference datasets
* Perform Integration or Batch correction using MNN correction.
* Support Multi-modal analysis for Cite-seq data
* Perform analysis on subsets (filter based on cell annotation)





## Steps for anlaysis using Seurat

* Setup the Seurat Object
* Standard pre-processing workflow
* Normalizing the data
* Identification of highly variable features (feature selection)
* Scaling the data
* Perform linear dimensional reduction
* Determine the ‘dimensionality’ of the dataset
* Cluster the cells
* Run non-linear dimensional reduction (UMAP/tSNE)
* Finding differentially expressed features (cluster biomarkers)
* Assigning cell type identity to clusters

##  Before we start

### Change Run type to R
If using google colab, change the colab runtype environment to R (default is python). Go to  Runtime -> Change runtime type -> In the "Notebook settings", change environment to "R"

### Mount google drive

In [ ]:
### Mount google drive
install.packages("googledrive") #only need to install occasionally install.packages(“httpuv”)
library("googledrive")

In [ ]:
#may update python version  #occasionally
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check)
  options(rlang_interactive=TRUE)
}

## Example using Seurat workflow in R
In this example, we will use the Seurat package (Link: https://satijalab.org/seurat/articles/get_started.html)

Citation for this tutorial: https://learn.gencore.bio.nyu.edu/single-cell-rnaseq/loading-your-own-data-in-seurat-reanalyze-a-different-dataset/


 ## Dataset
* The data is available by following THIS link.
* 1k Brain Cells from an E18 Mouse
* Chromium Demonstration (v2 Chemistry) Dataset by Cell Ranger 2.1.0
* Cells from a combined cortex, hippocampus and sub ventricular zone of an E18 mouse.
* 931 cells detected
* Sequenced on Illumina HiSeq2500 with approximately 56,000 reads per cell
26bp read1 (16bp Chromium barcode and 10bp UMI), 98bp read2 (transcript), and 8bp * I7 sample barcode
* Analysis run with —cells=2000.

### Note
Please download the input data files from either location listed above, and load it to your google drive folder `sample_data` that you can see on the left panel. Full path is `/content/sample_data/`

## Installation packages and dependencies
Takes several minutes to complete

In [ ]:
install.packages("Seurat")

library(dplyr)
library(Seurat)
library(patchwork)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘plyr’, ‘tensor’, ‘BH’, ‘sitmo’, ‘globals’, ‘listenv’, ‘parallelly’, ‘zoo’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’, ‘RcppTOML’, ‘here’, ‘gplots’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘sp’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘spatstat.data’, ‘spatstat.random’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘goftest’, ‘abind’, ‘deldir’, ‘polyclip’, ‘FNN’, ‘dqrng’, ‘cowplot’, ‘fitdistrplus’, ‘future’, ‘future.apply’, ‘ggrepel’, ‘ggridges’, ‘ica’, ‘igraph’, ‘irlba’, ‘leiden’, ‘lmtest’, ‘matrixStats’, ‘miniUI’, ‘patchwork’, ‘pbapply’, ‘plotly’, ‘png’, ‘progressr’, ‘RANN’, ‘Rcpp’, ‘RcppAnnoy’, ‘reticulate’, ‘ROCR’, ‘Rtsne’, ‘scattermore’, ‘sctransform’, ‘SeuratObject’, ‘shiny’, ‘spatstat.explore’, ‘spatstat.geom’, ‘uwot’, ‘RcppEigen’, ‘RcppProgress’




## Load input data

We start by reading in the data. The `Read10X()` function reads in the output of the cellranger pipeline from 10X, returning a **unique molecular identified (UMI) count matrix.**
The values in this matrix represent the number of molecules for each feature (i.e. gene; row) that are detected in each cell (column).

We next use the count matrix to create a **Seurat object.** The object serves as a container that contains both data (like the count matrix) and analysis (like PCA, or clustering results) for a single-cell dataset.  For example, the count matrix is stored in pbmc[["RNA"]]@counts.

* The `cellranger` pipeline from 10X takes the raw fastq files and processes it. For this tutorial, this step is already completed
* The `Read10X()` function reads in the output of the `cellranger` pipeline from 10X and returns a unique molecular identified (UMI) count matrix
* The values in this matrix represent the number of molecules for each feature (i.e. gene; row) that are detected in each cell (column).

## Important
Load input data to your google drive location. For this example, the path to input folder is `/content/sample_data/`. Can change path as needed in the URL below


In [ ]:
# Set the PBMC data directory
pbmc.data <- Read10X(data.dir = "/content/sample_data/raw_gene_bc_matrices/mm10/")

Learn more about this object

In [ ]:
#Learn more about this object
print("Dimensions")
dim(pbmc.data)

#The object serves as a container that contains both data and meta data
print("Structure")
str(pbmc.data)

## Initialize Seurat object and initial pre-filtering
Initialize the Seurat object with the raw (non-normalized count data)

Initial prefiltering is done at the gene and cell level by excluding any genes that are not expressed in at least 3 cells, and excluding any genes that do not have a minimum of 200 expressed genes in total.

Use the count matrix to create a Seurat object. The object serves as a container that contains both data (like the count matrix) and analysis (like PCA, or clustering results) for a single-cell dataset. For example, the count matrix is stored in pbmc[["RNA"]]@counts.

Note- Depending on your experiment and data, you might want to experiment with these cutoffs. For example, you might want to adjust the minimum number of detected genes to a higher threshold if you have deep coverage, or not impose it completely in case you have a very low number of reads for your cells.
[Reference: https://learn.gencore.bio.nyu.edu/single-cell-rnaseq/seurat-part-1-loading-the-data/]

In [ ]:
# Initialize the Seurat object with the raw (non-normalized data).
pbmc <- CreateSeuratObject(counts = pbmc.data, ##Unnormalized data such as raw counts or TPM
    project = "pbmc3k", #Sets the project name for the Seurat object
    min.cells = 3, #Include features detected in at least this many cells
     min.features = 200) ##Include cells where at least this many features

dim(pbmc)

## An object of class Seurat

Get more information

In [ ]:
# See the QC **metrics**
head(pbmc)

Lets examine a few genes in the first thirty cells

The "." values in the matrix represent 0s (no molecules detected). Since most values in an scRNA-seq matrix are 0, Seurat uses a sparse-matrix representation whenever possible. This results in significant memory and speed savings for Drop-seq/inDrop/10x data.

In [ ]:
# Lets examine a few genes in the first thirty cells
pbmc.data[c("Sox17", "Rp1"), 1:30]

#pbmc.data stored as sparse matrix
sparse.size <- object.size(pbmc.data)
sparse.size

## QC and selecting cells for further analysis

Seurat allows you to easily explore QC metrics and filter cells based on any user-defined criteria. A few QC metrics commonly used by the community include

* The number of unique genes detected in each cell
    * Low-quality cells or empty droplets will often have very few genes
    * Cell doublets or multiplets may exhibit an aberrantly high gene count
* Similarly, the total number of molecules detected within a cell (correlates strongly with unique genes)
* The percentage of reads that map to the mitochondrial genome
    * Low-quality / dying cells often exhibit extensive mitochondrial contamination
    * We calculate mitochondrial QC metrics with the PercentageFeatureSet() function, which calculates the percentage of counts originating from a set of features
    * We use the set of all genes starting with MT- as a set of mitochondrial genes


In [ ]:
# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
mito.genes <- grep(pattern = "^mt",
                   x = rownames(x = pbmc@meta.data),
                   ignore.case = TRUE, value = TRUE)

# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^mt-")

head(pbmc[["percent.mt"]])

Show QC metrics for the first 5 cells

In [ ]:
# Show QC metrics for the first 5 cells
head(pbmc@meta.data, 5)

### Visualize QC metrics

As a violin plot

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
        ncol = 3)

FeatureScatter is typically used to visualize feature-feature relationships, but can be used
for anything calculated by the object, i.e. columns in object metadata, PC scores etc.


In [ ]:
plot1 <- FeatureScatter(pbmc,
                        feature1 = "nCount_RNA",
                        feature2 = "percent.mt")
plot2 <- FeatureScatter(pbmc,
                        feature1 = "nCount_RNA",
                        feature2 = "nFeature_RNA")
plot1 + plot2

## Filtering

* We filter cells that have unique feature counts over 2,500 or less than 200
* We filter cells that have >5% mitochondrial counts




In [ ]:
pbmc2 <- subset(pbmc,
  subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
dim(pbmc2)
#number of samples went from

## Normalizing the data

After removing unwanted cells from the dataset, the next step is to normalize the data. By default, we employ a global-scaling normalization method “LogNormalize” that normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result. Normalized values are stored in pbmc[["RNA"]]@data.

In [ ]:
pbmc3 <- NormalizeData(pbmc2,
                      normalization.method = "LogNormalize",
                      scale.factor = 10000)

#same step for normlization as above
#pbmc <- NormalizeData(pbmc)

dim(pbmc3)
#14393   834

## Identification of highly variable features (feature selection)
We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The procedure in Seurat is described in detail here, and improves on previous versions by directly modeling the mean-variance relationship inherent in single-cell data, and is implemented in the FindVariableFeatures() function. By default, we return 2,000 features per dataset. These will be used in downstream analysis, like PCA.

In [ ]:
pbmc4 <- FindVariableFeatures(pbmc3,
                             selection.method = "vst",
                             nfeatures = 2000)

dim(pbmc4) #13714  2638

In [ ]:
# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(pbmc4), 10)

head(top10)

## Scaling the data
Next, we apply a linear transformation (‘scaling’) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The ScaleData() function:

* Shifts the expression of each gene, so that the mean expression across cells is 0
* Scales the expression of each gene, so that the variance across cells is 1
This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate
* The results of this are stored in pbmc[["RNA"]]@scale.data

In [ ]:
all.genes <- rownames(pbmc4)
pbmc5 <- ScaleData(pbmc4, features = all.genes)

Scaling is an essential step in the Seurat workflow, but only on genes that will be used as input to PCA. Therefore, the default in ScaleData() is only to perform scaling on the previously identified variable features (2,000 by default). To do this, omit the features argument in the previous function call, i.e.
`pbmc <- ScaleData(pbmc) #only 2000 genes`.

Your PCA and clustering results will be unaffected. However, Seurat heatmaps (produced as shown below with DoHeatmap()) require genes in the heatmap to be scaled, to make sure highly-expressed genes don’t dominate the heatmap. To make sure we don’t leave any genes out of the heatmap later, we are scaling all genes in this tutorial.

### To ‘regress out’ heterogeneity associated with (for example) cell cycle stage, or mitochondrial contamination.
For particularly for advanced users who would like to use this functionality, we strongly recommend the use of the new normalization workflow, SCTransform(). The method is described in the paper, with a separate vignette using Seurat v3 here. As with ScaleData(), the function SCTransform() also includes a vars.to.regress parameter. https://satijalab.org/seurat/articles/sctransform_vignette.html

In [ ]:
#remove unwanted sources of variation,
pbmc6 <- ScaleData(pbmc5, vars.to.regress = "percent.mt")

## Perform linear dimensional reduction
Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
pbmc7 <- RunPCA(pbmc6, features = VariableFeatures(object = pbmc6))

#Seurat provides several useful ways of visualizing both cells and features that define the PCA, including VizDimReduction(), DimPlot(), and DimHeatmap()
# Examine and visualize PCA results a few different ways
print(pbmc7[["pca"]], dims = 1:5, nfeatures = 5)
VizDimLoadings(pbmc7, dims = 1:2, reduction = "pca")

DimPlot(pbmc7, reduction = "pca")

DimHeatmap() allows for easy exploration of the primary sources of heterogeneity in a dataset, and can be useful when trying to decide which PCs to include for further downstream analyses. Both cells and features are ordered according to their PCA scores. Setting cells to a number plots the ‘extreme’ cells on both ends of the spectrum, which dramatically speeds plotting for large datasets. Though clearly a supervised analysis, we find this to be a valuable tool for exploring correlated feature sets.


In [ ]:
DimHeatmap(pbmc7, dims = 1:15, cells = 500, balanced = TRUE)

## Determine the ‘dimensionality’ of the dataset
To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a ‘metafeature’ that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many components should we choose to include? 10? 20? 100?

In Macosko et al, we implemented a resampling test inspired by the JackStraw procedure. We randomly permute a subset of the data (1% by default) and rerun PCA, constructing a ‘null distribution’ of feature scores, and repeat this procedure. We identify ‘significant’ PCs as those who have a strong enrichment of low p-value features.

NOTE: This process can take a long time for big datasets, comment out for expediency. More approximate techniques such as those implemented in ElbowPlot() can be used to reduce computation time

```
#pbmc8 <- JackStraw(pbmc7, num.replicate = 100)
#pbmc8 <- ScoreJackStraw(pbmc8, dims = 1:20)
```



### Elow plot
An alternative heuristic method generates an ‘Elbow plot’: a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot() function). In this example, we can observe an ‘elbow’ around PC9-10, suggesting that the majority of true signal is captured in the first 12 PCs.



In [ ]:
# Elbow plot
ElbowPlot(pbmc7)

Identifying the true dimensionality of a dataset – can be challenging/uncertain for the user. We therefore suggest these three approaches to consider.
* The first is more supervised, exploring PCs to determine relevant sources of heterogeneity, and could be used in conjunction with GSEA for example.
* The second implements a statistical test based on a random null model, but is time-consuming for large datasets, and may not return a clear PC cutoff.
* The third is a heuristic that is commonly used, and can be calculated instantly.

In this example, all three approaches yielded similar results, but we might have been justified in choosing anything between PC 7-12 as a cutoff. We chose 10 PCs here

The Seurat authors advise users to err on the higher side when choosing this parameter. For example, performing downstream analyses with only 5 PCs does significantly and adversely affect results.

## Cluster the cells
Seurat v3 applies a graph-based clustering approach, building upon initial strategies
in (Macosko et al). Importantly, the distance metric which drives the clustering analysis
(based on previously identified PCs) remains the same. However, our approach to partitioning
the cellular distance matrix into clusters has dramatically improved. The approach was heavily
inspired by recent manuscripts which applied graph-based clustering approaches to scRNA-seq data
Briefly, these methods embed cells in a graph structure - for example a **K-nearest neighbor
(KNN) graph**, with edges drawn between cells with similar feature expression patterns, and
then attempt to partition this graph into highly interconnected ‘quasi-cliques’ or
‘communities’.

* As in PhenoGraph, we first construct a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods **(Jaccard similarity)**. This step is performed using the `FindNeighbors()` function, and takes as input the previously defined dimensionality of the dataset (first 10 PCs).
* To cluster the cells, we next apply modularity optimization techniques such as the Louvain algorithm (default) or SLM [SLM, Blondel et al., Journal of Statistical Mechanics], to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters()` function implements this procedure, and contains a resolution parameter that sets the ‘granularity’ of the downstream clustering, with increased values leading to a greater number of clusters. Setting this parameter between 0.4-1.2 typically returns good results for single-cell datasets of around 3K cells. Optimal resolution often increases for larger datasets. The clusters can be found using the `Idents()` function.


In [ ]:
numberOfDims = 12

pbmc8 <- FindNeighbors(pbmc7, dims = 1:numberOfDims)
pbmc8 <- FindClusters(pbmc8, resolution = 0.5)
#Number of nodes: 2638
#Number of edges: 95905

# Look at cluster IDs of the first 5 cells
head(Idents(pbmc8), 5)

## Run non-linear dimensional reduction (UMAP/tSNE)
Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, we suggest using the same PCs as input to the clustering analysis.


In [ ]:
# If you haven't installed UMAP, you can do so via reticulate::py_install(packages =
#'umap-learn')
pbmc9 <- RunUMAP(pbmc8, dims = 1:numberOfDims)

# note that you can set `label = TRUE` or use the LabelClusters function to help label
#individual clusters
DimPlot(pbmc9, reduction = "umap")

#You can save the object at this point so that it can easily be loaded back in without
#having to rerun the computationally intensive steps performed above, or easily shared with collaborators.

saveRDS(pbmc9, file = "/content/sample_data/Seurat_tutorial2.rds")

## Finding differentially expressed features (cluster biomarkers)
Seurat can help find markers that define clusters via differential expression. By default, it identifies positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. FindAllMarkers() automates this process for all clusters, but you can also test groups of clusters vs. each other, or against all cells.

The min.pct argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the thresh.test argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, max.cells.per.ident can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significant and the most highly differentially expressed features will likely still rise to the top.

Find markers for every cluster compared to all remaining cells, report only the positive ones


In [ ]:
pbmc.markers <- FindAllMarkers(pbmc9,
                               only.pos = TRUE,
                               min.pct = 0.25,
                               logfc.threshold = 0.25)

de.markers <- pbmc.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC)

head(de.markers)
write.csv(de.markers, file = "/content/sample_data/Tutorial2_demarkers.csv")


Seurat has several tests for differential expression which can be set with the test.use parameter (see our DE vignette for details). For example, the ROC test returns the ‘classification power’ for any individual marker (ranging from 0 - random, to 1 - perfect).

In [ ]:
cluster0.markers <- FindMarkers(pbmc9,
                                ident.1 = 0,
                                logfc.threshold = 0.25,
                                test.use = "roc",
                                only.pos = TRUE)
head(cluster0.markers)
write.csv(de.markers, file = "/content/sample_data/Tutorial2_classif.power.csv")

## Visulize markers
There are several tools for visualizing marker expression.
* VlnPlot() (shows expression probability distributions across clusters), and * FeaturePlot() (visualizes feature expression on a tSNE or PCA plot) are the most commonly used visualizations.
* RidgePlot(), CellScatter(), and DotPlot()

DoHeatmap() generates an expression heatmap for given cells and features. In this case, we are plotting the top 20 markers (or all markers if less than 20) for each cluster.


In [ ]:
pbmc.markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10

DoHeatmap(pbmc9, features = top10$gene) + NoLegend()

In [ ]:
#Write session info
sessionInfo()


## Tutorial complete
The tutorial is now complete. Please download the two RDS files exported to the output directory '/content/sample_data/', and the images generated as they will not be stored permanently



## References
* https://satijalab.org/seurat/articles/pbmc3k_tutorial.html
* https://learn.gencore.bio.nyu.edu/single-cell-rnaseq/loading-your-own-data-in-seurat-reanalyze-a-different-dataset/